# ¿Cómo se relacionan los rendimientos y la volatilidad de las acciones de energía y el mercado en general?

## Introduccion

**Contexto empresarial.** Usted es analista en un gran banco centrado en inversiones en acciones de recursos naturales. Recientemente realizó un análisis de las siguientes acciones de energía y cómo su volumen de operaciones está relacionado con su volatilidad:

1. Dominion Energy Inc. (Símbolo bursátil: D)
2. Exelon Corp. (Símbolo bursátil: EXC)
3. NextEra Energy Inc. (Símbolo bursátil: NEE)
4. Southern Co. (Símbolo bursátil: SO)
5. Duke Energy Corp. (Símbolo bursátil: DUK)

Su jefe estaba bastante complacido con su análisis anterior y ahora quiere que realice un análisis adicional para que pueda averiguar cómo dimensionar las posiciones potenciales en estas acciones... es decir, qué porcentaje de la cartera de inversión debe dedicarse a cada una de estas acciones. Específicamente, quiere que observe los rendimientos diarios y la volatilidad de cada acción, así como del mercado en general (es decir, no solo del sector energético).

Esto es importante porque la alta volatilidad implica un mayor riesgo, y a su jefe le gustaría saber si los rendimientos potenciales de estas acciones de energía de alta volatilidad lo compensan por el riesgo adicional. Además, debido a que su rendimiento se mide o se compara con el mercado en general, quiere saber si estas acciones generalmente superan al mercado en general.

**Problema empresarial.** Con base en el contexto anterior, su jefe le planteó la siguiente pregunta: **"¿Cuál es la relación entre la volatilidad diaria y los rendimientos de estas acciones, y cuál es la relación entre los rendimientos diarios de estas acciones y el mercado de valores en general?"**

**Contexto analítico.** Los datos que se le proporcionaron están en formato de valores separados por comas (CSV) y comprenden datos de precios y volumen de negociación para las acciones anteriores. Procederá: (1) realizando una limpieza preliminar de los datos; (2) crear características adicionales requeridas para nuestro análisis; (3) etiquetar los datos en grupos de volatilidad, o regímenes, y determinar cómo se relaciona la volatilidad con los rendimientos; y finalmente (4) comparar estos rendimientos con los del mercado más amplio.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/MyDrive/Curso Data Science para todos/Datos/'

/content/gdrive/MyDrive/Curso Data Science para todos/Datos


## Limpieza preliminar de datos

Antes de que podamos continuar con el análisis y el modelado de datos, primero debemos determinar si los datos relevantes son adecuados para continuar tal como están o si necesitan una limpieza adicional. En este caso, hemos recibido un archivo de valores separados por comas (CSV) que incluye los siguientes datos:

1. **Fecha:** El día del año
2. **Apertura:** El precio de apertura de las acciones del día
3. **Alto:** El precio bursátil más alto observado del día
4. **Mínimo:** El precio bursátil observado más bajo del día
5. **Cierre:** El precio de cierre de la acción del día
6. **Adj Close:** El precio de cierre de acciones ajustado para el día (ajustado por splits y dividendos)
7. **Volumen:** El volumen de las acciones negociadas durante el día.
8. **Símbolo:** El símbolo de esa acción en particular

Un problema muy común que surge en los conjuntos de datos son los valores faltantes. Veamos cómo identificar si nuestro conjunto de datos tiene este problema o no, y cómo lidiar con los valores faltantes.

In [4]:
raw_df = pd.read_csv('EnergySectorData.csv')
raw_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D


In [5]:
raw_df.shape

(6275, 8)

In [6]:
raw_df.Symbol.unique()

array(['D', 'EXC', 'NEE', 'SO', 'DUK'], dtype=object)

Empecemos por determinar si nos faltan valores. Podemos usar el método DataFrame ```pandas``` ```isnull()``` para comprobar los valores ```NaN``` en ```raw_df``` (es decir, comprobar los valores nulos):

In [7]:
raw_df.isnull().sum()

Date          0
Open          2
High         14
Low           7
Close         7
Adj Close     7
Volume       22
Symbol        0
dtype: int64

Aquí vemos que tenemos algunos valores faltantes. En su lugar, usemos el método ```mean()``` para determinar qué porcentaje de cada columna nos falta.

In [8]:
raw_df.isnull().mean()

Date         0.000000
Open         0.000319
High         0.002231
Low          0.001116
Close        0.001116
Adj Close    0.001116
Volume       0.003506
Symbol       0.000000
dtype: float64

Vemos aquí que nos falta menos del 0,5% de las observaciones en cualquier columna dada.



### Ejercicio 1:
No queremos ningún valor faltante en nuestro análisis. ¿Cuál de las siguientes opciones es la PEOR opción sobre cómo proceder en este caso?

(a) Complete el valor faltante de cualquier día con el valor del día anterior

(b) Reemplace los valores faltantes volviendo a recopilar los datos

(c) Estime los valores faltantes interpolándolos a partir de los valores de otros puntos de datos similares

(d) Eliminar filas del conjunto de datos que contienen valores faltantes

**Respuesta.** (a). Tenemos un par de opciones que generalmente son aceptables sobre cómo proceder con los valores faltantes:

1. Una opción es reemplazar los valores faltantes volviendo a recopilar los datos. Sin embargo, esta opción suele ser bastante costosa en horas-hombre reales, por lo que la dejaremos de lado por ahora.

2. Otra opción es tratar de estimar los valores faltantes usando algún método de estimación razonable, interpolando desde otros puntos de datos. Sin embargo, esto puede ser complicado y dado que falta una cantidad tan pequeña de nuestros datos, renunciaremos a esta opción.

3. En la práctica, una opción elegida regularmente cuando solo falta una pequeña cantidad de datos es simplemente eliminar las filas que tienen datos faltantes. Por lo general, esta opción es adecuada siempre que los datos eliminados sean una parte insignificante de los datos en estudio. Aquí elegiremos esta opción ya que simplifica el análisis y no debería dañar ningún resultado en el futuro.

La respuesta (a) es problemática porque reemplazar un valor faltante con el valor del día anterior no tiene sentido para las acciones porque se sabe que los precios de las acciones y los volúmenes de negociación se mueven día a día en lugar de permanecer sin cambios durante un período prolongado de tiempo. Dado que nos ocuparemos de los rendimientos diarios y la volatilidad, esto es especialmente problemático, ya que la volatilidad de cualquier día faltante y el retorno predeterminados son 0.

Tenga en cuenta que estas opciones para la limpieza de datos deben sopesarse cuidadosamente al comenzar un nuevo estudio de ciencia de datos.

In [9]:
progress_df = raw_df.dropna()
progress_df.shape

(6228, 8)

## Estamdarizar fechas

Como se discutió anteriormente, podemos eliminar los valores faltantes ya que no faltan muchas muestras, y no se espera que la eliminación de una pequeña cantidad de fechas afecte significativamente el análisis:

In [10]:
progress_df = raw_df.dropna().copy()

In [11]:
progress_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1230,1230,1230,1230,1230,1230,1230
DUK,1249,1249,1249,1249,1249,1249,1249
EXC,1239,1239,1239,1239,1239,1239,1239
NEE,1251,1251,1251,1251,1251,1251,1251
SO,1259,1259,1259,1259,1259,1259,1259


Aún así, vemos que diferentes símbolos tienen diferentes números de fechas. Nos gustaría que todos los símbolos tuvieran el mismo conjunto de fechas para propósitos de análisis. Vamos a crear un nuevo ```clean_df``` que corresponda a un DataFrame con el mismo número de filas para cada ```Símbolo```, donde todos los símbolos comparten el mismo conjunto de fechas:

In [12]:
set(progress_df[progress_df['Symbol'] == 'D']['Date'][0:5])

{'2014-07-28', '2014-07-29', '2014-07-30', '2014-07-31', '2014-08-01'}

In [13]:
set_dates_D = set(progress_df[progress_df['Symbol'] == 'D']['Date'])
set_dates_EXC = set(progress_df[progress_df['Symbol'] == 'EXC']['Date'])
set_dates_NEE = set(progress_df[progress_df['Symbol'] == 'NEE']['Date'])
set_dates_SO = set(progress_df[progress_df['Symbol'] == 'SO']['Date'])
set_dates_DUK = set(progress_df[progress_df['Symbol'] == 'DUK']['Date'])
set_unique_dates = set.intersection(set_dates_D,set_dates_EXC,set_dates_NEE,set_dates_SO,set_dates_DUK)

In [14]:
# Filtrar el nuevo df solo para las fechas que están presentes en cada símbolo (es decir, las fechas superpuestas)
clean_df = progress_df[progress_df['Date'].isin(set_unique_dates)].copy()

In [15]:
clean_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1192,1192,1192,1192,1192,1192,1192
DUK,1192,1192,1192,1192,1192,1192,1192
EXC,1192,1192,1192,1192,1192,1192,1192
NEE,1192,1192,1192,1192,1192,1192,1192
SO,1192,1192,1192,1192,1192,1192,1192


In [18]:
clean_df.shape

(5960, 8)

Ahora vemos que cada símbolo tiene el mismo número de fechas únicas. Escribamos un programa de verificación rápida para asegurarnos de que el ```clean_df``` resultante tenga las mismas fechas para todos los símbolos.

### Ejercicio 2:
Escriba código para asegurarse de que cada uno de los símbolos comparta el mismo conjunto de fechas únicas. (Sugerencia: utilice el método ```set()```.)

In [19]:
check_set_dates_D = set(clean_df[clean_df['Symbol'] == 'D']['Date'])
check_set_dates_EXC = set(clean_df[clean_df['Symbol'] == 'EXC']['Date'])
check_set_dates_NEE = set(clean_df[clean_df['Symbol'] == 'NEE']['Date'])
check_set_dates_SO = set(clean_df[clean_df['Symbol'] == 'SO']['Date'])
check_set_dates_DUK = set(clean_df[clean_df['Symbol'] == 'DUK']['Date'])

print(check_set_dates_D == check_set_dates_EXC)
print(check_set_dates_D == check_set_dates_NEE)
print(check_set_dates_D == check_set_dates_SO)
print(check_set_dates_D == check_set_dates_DUK)

True
True
True
True


Ahora que hemos completado la limpieza preliminar de los datos, avancemos con la determinación de las relaciones entre: (1) rendimientos de acciones y volatilidad, y (2) rendimientos de acciones y rendimientos de mercado más amplios.

## Agregar variables adicionales requeridas para nuestro análisis

Recuerde que la pregunta original requiere que investiguemos tanto los rendimientos diarios de las acciones como la volatilidad de esos rendimientos. Esto significa que las medidas importantes de interés son:

1. Rendimiento diario de acciones (abierto a cerrado)
2. Volatilidad de la rentabilidad diaria de las acciones

¿Por qué cada uno de estos es importante?

1. Volatilidad: brinda información sobre la cantidad de movimiento de precios en un día determinado. La volatilidad está directamente relacionada con el nivel de riesgo involucrado en la tenencia de acciones.
2. Retorno: Nos da una idea del retorno de la inversión durante un período de tiempo.

Calculemos estas estadísticas y agréguelas al DataFrame ```clean_df```:

In [20]:
clean_df['VolStat'] = (clean_df['High'] - clean_df['Low']) / clean_df['Open']
clean_df['Return'] = (clean_df['Close'] / clean_df['Open']) - 1.0
clean_df['Volume_Millions'] = clean_df['Volume'] / 1000000.0 #
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018


Aquí vemos que hemos agregado tres columnas a ```clean_df```, a saber, ```VolStat```, ```Return``` y ```Volume_Millions``` (la última es simplemente por conveniencia, ya que los valores en la columna ```Volumen``` son bastante grandes).

Dado que buscamos analizar la relación entre la volatilidad diaria y los rendimientos, una columna adicional que tiene sentido agregar es una que dice ```Verdadero``` cuando el rendimiento diario es positivo, y ```Falso``` cuando el rendimiento diario negativo. Luego podemos agrupar los días en cohortes de rendimiento positivo y negativo y comparar la volatilidad promedio en esos días. Llamaremos a esta columna ```ReturnFlag```.

Podemos lograr esto usando una **función anónima**; es decir, una función que está definida pero no nombrada:

```python
lambda arguments: expression
```

La palabra clave ```lambda``` le dice a Python que estamos usando una función anónima. A continuación, los ```argumentos``` son el nombre que le damos a las entradas. Puede ser ```x```, o ```y```, o como quiera llamarlo el usuario. En este caso, usaremos el nombre ```row``` para el nombre del argumento de entrada, ya que la entrada será una fila de un DataFrame. La ```expresión``` es lo que luego se aplica a los ```argumentos```; esta es la función.

Echemos un vistazo a cómo podemos usar funciones anónimas para crear la función ```ReturnFlag```:

In [22]:
clean_df['ReturnFlag'] = clean_df.apply(lambda x: True if x['Return'] > 0 else False, axis=1) #
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True


Observe que el método ```apply()``` toma una función anónima y la aplica a las filas del DataFrame mediante el uso del segundo argumento ```axis```. ```axis=0``` aplica la función a las columnas, mientras que ```axis=1``` aplica la función a las filas.

Entonces, ¿qué está sucediendo en la siguiente declaración?
```python
clean_df['ReturnFlag'] = clean_df.apply(lambda row: True if row['Return] > 0 else False, axis=1)
```

1. ```pandas``` reconoció a través del método ```apply``` que está operando en ```clean_df``` DataFrame
2. El método ```apply``` toma una función como entrada que se aplicará al DataFrame ```clean_df```
3. Dado que el segundo argumento de ```apply``` es ```axis=1```, la entrada en la función anónima es una sola fila
4. Para cada fila, ```row['Return']``` devuelve el valor ```Return``` para esa fila, y posteriormente se pasa a través de la instrucción if, devolviendo True si es mayor que cero y False de lo contrario
5. El nuevo valor se almacena en la columna ```clean_df['ReturnFlag']```

### Ejercicio 3:

Usando ```apply()``` y ```lambda```, escriba código para crear una nueva columna llamada ```YYYY``` a ```clean_df```, donde la nueva columna es el año de la observación como una cadena. Por ejemplo, si el valor de la fila ```Date``` es 2014-07-28, entonces el valor en la nueva columna para el año sería '2014'. Recuerda que puedes acceder a los primeros 4 caracteres de alguna cadena ```my_string``` usando ```my_string[:4]```.

In [23]:
clean_df['YYYY'] = clean_df.apply(lambda row: row['Date'][0:4], axis=1)
clean_df

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6270,2019-07-22,88.599998,88.690002,87.839996,88.150002,88.150002,2160300.0,DUK,0.009594,-0.005079,2.1603,False,2019
6271,2019-07-23,88.180000,88.239998,87.370003,87.680000,87.680000,3141800.0,DUK,0.009866,-0.005670,3.1418,False,2019
6272,2019-07-24,87.949997,87.980003,86.800003,87.180000,87.180000,2386500.0,DUK,0.013417,-0.008755,2.3865,False,2019
6273,2019-07-25,86.900002,87.389999,86.169998,86.489998,86.489998,2680300.0,DUK,0.014039,-0.004718,2.6803,False,2019


Avancemos con el etiquetado de los regímenes de volatilidad presentes en los datos: estos regímenes son útiles para desglosar el análisis de rendimiento de las acciones por períodos de volatilidad baja, media y alta. Permitirá un análisis más granular que solo mirar los promedios generales sin un desglose.

### Ejercicio 4:
Usando ```apply()``` y ```lambda```, escriba un script para crear una nueva columna en ```clean_df``` llamada ```AvgDailyPrice``` que calcula un precio promedio diario basado en de si el volumen diario supera o no los 5 millones. Establezca el valor de la nueva columna en (Open + High + Low + Close)/4 si el volumen es mayor que 5 millones, o establezca el valor en (High + Low + Close)/3 si el volumen es menor o igual a 5 millones.



In [25]:
clean_df['AvgDailyPrice'] = clean_df.apply(lambda row:(row['Open']+row['High']+row['Low']+row['Close'])/4 if row['Volume'] > 5000000 
                                           else (row['High']+row['Low']+row['Close'])/3, axis=1)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000


## Etiquetado de los regímenes de volatilidad del sector energético



```python
if VolStrat > upper_threshold_dict['D']:
    VolLevel = '3_HIGH'
elif VolStrat < lower_threshold_dict['D']:
    VolLevel = '1_LOW'
else:
    VolLevel = '2_MEDIUM'
```


In [26]:
lower_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.25).to_dict() # 
lower_threshold_dict

{'D': 0.010240046986389131,
 'DUK': 0.010018315803797114,
 'EXC': 0.011881680089172456,
 'NEE': 0.010258642787424542,
 'SO': 0.009734019893739591}

In [27]:
upper_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.75).to_dict() # 
upper_threshold_dict

{'D': 0.017960914526108228,
 'DUK': 0.01759838077408522,
 'EXC': 0.021801523265676366,
 'NEE': 0.01768021802425081,
 'SO': 0.01683044706857934}

Una vez más, nuestro objetivo es etiquetar los períodos de volatilidad baja, media y alta. Definamos una nueva columna llamada ```VolLevel``` para cada símbolo usando algunos valores de límite inferior y superior. Definamos una función personalizada que se aplicará a cada fila para lograr este objetivo.

In [28]:
def my_custom_row_function(row):
    row_symbol = row['Symbol']    # 
    row_volstat = row['VolStat']  # 
    
    lower_threshold = lower_threshold_dict[row_symbol] # Dictionary  {string:float}
    upper_threshold = upper_threshold_dict[row_symbol] # Dictionary  {string:float}
    
    # La decisión de la función, el valor devuelto dependiendo de la volatilidad baja, media o alta
    if row_volstat > upper_threshold:
        return '3_HIGH'
    elif row_volstat < lower_threshold:
        return '1_LOW'
    else:
        return '2_MEDIUM'

Ahora apliquemos la función a cada fila del DataFrame ```clean_df``` usando una declaración ```lambda```. Almacenamos los valores devueltos en la nueva columna ```VolLevel```:

In [29]:
clean_df['VolLevel'] = clean_df.apply(lambda row: my_custom_row_function(row), axis=1)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332,3_HIGH
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001,2_MEDIUM
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336,3_HIGH
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333,3_HIGH
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000,2_MEDIUM


Si bien el flujo de trabajo aquí puede parecer complejo al principio, la capacidad de aplicar funciones personalizadas, agrupar por ciertas funciones y construir estadísticas de resumen resultará invaluable a medida que avanza hacia análisis más avanzados.

### Ejercicio 5:

Usando ```clean_df``` y una instrucción ```lambda``` dentro de ```apply()```, escriba una función ```new_custom_function()``` y un script para agregar una nueva columna a el DataFrame (llámelo ```EnhancedVolLevel```) que funciona de manera similar a VolLevel pero en su lugar proporciona cinco categorías de nivel de volatilidad utilizando la siguiente lógica para determinar la etiqueta del nivel de volatilidad:

```python
if VolStrat > 90th percentile:
    VolLevel = '5_VERY_HIGH'
elif VolStrat > 75th percentile:
    VolLevel = '4_HIGH'
elif VolStrat > 25th percentile:
    VolLevel = '3_MEDIUM'
elif VolStrat > 10th percentile:
    VolLevel = '2_LOW'
else:
    VolLevel = '1_VERY_LOW'
```

Recuerde que cada percentil debe calcularse por símbolo. Utilice estas nuevas etiquetas para ver si hay algún patrón entre los niveles de volatilidad y la dirección de los rendimientos. Produzca el DataFrame para poder ejecutar el siguiente comando:

```python
clean_df.groupby(['Symbol','EnhancedVolLevel'])['ReturnFlag'].mean()
```

In [30]:
def new_custom_function(row):
    row_symbol = row['Symbol']    #
    row_volstat = row['VolStat']  # 
    
    very_lower_threshold = very_lower_threshold_dict[row_symbol] # Dictionary  {string:float}
    lower_threshold = lower_threshold_dict[row_symbol] # Dictionary {string:float}
    upper_threshold = upper_threshold_dict[row_symbol] # Dictionary {string:float}
    very_upper_threshold = very_upper_threshold_dict[row_symbol] # Dictionary {string:float}
    
    if row_volstat > very_upper_threshold:
        return '5_VERY_HIGH'
    elif row_volstat > upper_threshold:
        return '4_HIGH'
    elif row_volstat > lower_threshold:
        return '3_MEDIUM'
    elif row_volstat > very_lower_threshold:
        return '2_LOW'
    else:
        return '1_VERY_LOW'
    
#  thresholds
very_upper_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.90).to_dict() # 90th percentile bound
upper_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.75).to_dict() # 75th percentile bound
lower_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.25).to_dict() # 25th percentile bound
very_lower_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.10).to_dict() # 10th percentile bound

In [31]:
clean_df['EnhancedVolLevel'] = clean_df.apply(lambda row: new_custom_function(row), axis=1)
clean_df

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel,EnhancedVolLevel
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332,3_HIGH,4_HIGH
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001,2_MEDIUM,3_MEDIUM
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336,3_HIGH,5_VERY_HIGH
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333,3_HIGH,4_HIGH
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000,2_MEDIUM,3_MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6270,2019-07-22,88.599998,88.690002,87.839996,88.150002,88.150002,2160300.0,DUK,0.009594,-0.005079,2.1603,False,2019,88.226667,1_LOW,2_LOW
6271,2019-07-23,88.180000,88.239998,87.370003,87.680000,87.680000,3141800.0,DUK,0.009866,-0.005670,3.1418,False,2019,87.763334,1_LOW,2_LOW
6272,2019-07-24,87.949997,87.980003,86.800003,87.180000,87.180000,2386500.0,DUK,0.013417,-0.008755,2.3865,False,2019,87.320002,2_MEDIUM,3_MEDIUM
6273,2019-07-25,86.900002,87.389999,86.169998,86.489998,86.489998,2680300.0,DUK,0.014039,-0.004718,2.6803,False,2019,86.683332,2_MEDIUM,3_MEDIUM


In [32]:
# Resultado
clean_df.groupby(['Symbol','EnhancedVolLevel'])['ReturnFlag'].mean()

Symbol  EnhancedVolLevel
D       1_VERY_LOW          0.508333
        2_LOW               0.573034
        3_MEDIUM            0.558725
        4_HIGH              0.528090
        5_VERY_HIGH         0.408333
DUK     1_VERY_LOW          0.550000
        2_LOW               0.522472
        3_MEDIUM            0.558725
        4_HIGH              0.511236
        5_VERY_HIGH         0.433333
EXC     1_VERY_LOW          0.533333
        2_LOW               0.533708
        3_MEDIUM            0.508389
        4_HIGH              0.533708
        5_VERY_HIGH         0.500000
NEE     1_VERY_LOW          0.566667
        2_LOW               0.550562
        3_MEDIUM            0.567114
        4_HIGH              0.522472
        5_VERY_HIGH         0.500000
SO      1_VERY_LOW          0.516667
        2_LOW               0.511236
        3_MEDIUM            0.530201
        4_HIGH              0.500000
        5_VERY_HIGH         0.516667
Name: ReturnFlag, dtype: float64

Vemos que la volatilidad y los rendimientos de las acciones no exhiben patrones sólidos en términos de la dirección de rendimiento promedio (positiva o negativa) para un régimen de volatilidad determinado.

## Comparación de los rendimientos de las acciones con rendimientos de mercado más amplios

Ahora, echemos un vistazo a la segunda parte de la pregunta de su jefe: **¿cuál es la relación entre los rendimientos más amplios del mercado y los rendimientos de estas cinco acciones energéticas?** El **índice S&P 500** es un índice bursátil compuesto por unas 500 empresas estadounidenses públicas de gran capitalización. El índice se usa a menudo como una representación del mercado de valores de EE. UU. Si podemos determinar si existe o no una fuerte relación entre los rendimientos de estas 5 acciones de energía y los del índice S&P 500, podemos determinar si hay características idiosincrásicas significativas en juego entre los rendimientos de las acciones del sector energético, o si los rendimientos son impulsado únicamente por el mercado más amplio.

Los rendimientos de mercado del ETF (fondo cotizado en bolsa) del índice S&P 500 negociable están disponibles en ```SPY.csv``` (el símbolo de "acción" del ETF es SPY). Carguemos los datos y agreguemos los rendimientos diarios a los datos del sector de energía limpia:

In [33]:
market_df = pd.read_csv('SPY.csv')
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000


In [34]:
market_df['Symbol'] = 'SPY' # 
market_df['Return'] = (market_df['Close'] / market_df['Open']) - 1.0 # 
market_df['VolStat'] = (market_df['High'] - market_df['Low']) / market_df['Open']
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Return,VolStat
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000,SPY,0.002846,0.003862
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000,SPY,0.002780,0.005560
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000,SPY,0.004038,0.005451
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000,SPY,0.002059,0.004169
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000,SPY,-0.000752,0.004766


In [35]:
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel,EnhancedVolLevel
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332,3_HIGH,4_HIGH
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001,2_MEDIUM,3_MEDIUM
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336,3_HIGH,5_VERY_HIGH
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333,3_HIGH,4_HIGH
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000,2_MEDIUM,3_MEDIUM


Nos gustaría fusionar los rendimientos del mercado en ```market_df``` y los datos de acciones de energía en ```clean_df```. Esto se puede lograr usando ```pd.merge()``` - un método versátil para unir tramas de datos.

Para aquellos de ustedes familiarizados con SQL, la fusión y unión de DataFrames se puede lograr de la misma manera que SQL realiza estas tareas (si no está familiarizado con SQL, no se preocupe, lo cubriremos en casos posteriores). En este caso, nos gustaría usar la intersección de fechas en fechas ```clean_df``` y ```market_df``` como índices en la fusión (es decir, en lenguaje SQL, realizaremos un ``` combinación ``` interna):

In [36]:
merged_df = pd.merge(clean_df, market_df[['Date','Return']], how='inner', on='Date', suffixes=('','_SPY'))
merged_df

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel,EnhancedVolLevel,Return_SPY
0,2014-08-18,68.970001,69.250000,68.559998,68.680000,56.164867,1375600.0,D,0.010004,-0.004205,1.3756,False,2014,68.829999,1_LOW,2_LOW,0.002846
1,2014-08-18,31.930000,32.099998,31.730000,31.820000,26.593485,4037400.0,EXC,0.011588,-0.003445,4.0374,False,2014,31.883333,1_LOW,2_LOW,0.002846
2,2014-08-18,96.589996,97.180000,95.889999,96.139999,82.166077,1098200.0,NEE,0.013355,-0.004659,1.0982,False,2014,96.403333,2_MEDIUM,3_MEDIUM,0.002846
3,2014-08-18,43.529999,43.720001,43.279999,43.380001,34.210857,2979800.0,SO,0.010108,-0.003446,2.9798,False,2014,43.460000,2_MEDIUM,3_MEDIUM,0.002846
4,2014-08-18,72.300003,72.650002,71.919998,72.089996,58.080738,1826000.0,DUK,0.010097,-0.002905,1.8260,False,2014,72.219999,2_MEDIUM,3_MEDIUM,0.002846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5885,2019-07-26,74.730003,75.349998,74.610001,75.150002,75.150002,3076500.0,D,0.009902,0.005620,3.0765,True,2019,75.036667,1_LOW,2_LOW,0.004156
5886,2019-07-26,46.240002,46.480000,45.810001,45.820000,45.820000,5525900.0,EXC,0.014490,-0.009083,5.5259,False,2019,46.087501,2_MEDIUM,3_MEDIUM,0.004156
5887,2019-07-26,208.139999,209.750000,207.800003,209.169998,209.169998,1081200.0,NEE,0.009369,0.004949,1.0812,True,2019,208.906667,1_LOW,2_LOW,0.004156
5888,2019-07-26,55.340000,55.830002,55.200001,55.740002,55.740002,2288700.0,SO,0.011384,0.007228,2.2887,True,2019,55.590002,2_MEDIUM,3_MEDIUM,0.004156


### Ejercicio 6:

Usando solo el Símbolo D en ```clean_df``` y ```market_df```, use ```pd.merge()``` para determinar cuántas fechas hay en ```clean_df``` que no están en ```market_df```. Además, ¿cuántas fechas hay en ```market_df``` que no están en ```clean_df```? (Sugerencia: isnull() puede ser útil para simplificar la solución).

In [37]:
outer_df = pd.merge(clean_df[clean_df['Symbol'] == 'D'][['Date','Return']], market_df[['Date','Return']], how='outer', on='Date', suffixes=('','_SPY'))
print('--- outer merge HEAD ---')
print(outer_df.head())
print('--- outer merge TAIL ---')
print(outer_df.tail())
print('--- outer merge NaN count ---')
outer_df.isnull().sum()

--- outer merge HEAD ---
         Date    Return  Return_SPY
0  2014-07-28  0.016201         NaN
1  2014-07-29 -0.010471         NaN
2  2014-07-30 -0.014714         NaN
3  2014-07-31 -0.014425         NaN
4  2014-08-01  0.003861         NaN
--- outer merge TAIL ---
            Date  Return  Return_SPY
1268  2019-08-12     NaN   -0.006518
1269  2019-08-13     NaN    0.016716
1270  2019-08-14     NaN   -0.014476
1271  2019-08-15     NaN   -0.000807
1272  2019-08-16     NaN    0.008273
--- outer merge NaN count ---


Date           0
Return        81
Return_SPY    14
dtype: int64

### Ejercicio 7:

Use ```market_df``` y ```clean_df``` para crear un nuevo DataFrame ```modified_clean_df``` que es igual que ```clean_df``` pero con una nueva columna llamada ```MeanMonthSPYReturn ```. Cada valor en la nueva columna debe ser el rendimiento mensual medio de SPY para el mes dado de la ```Fecha``` de cada fila. La salida para la primera y la última fila del marco de datos ```modified_clean_df``` debería producir la siguiente salida de head(1) y tail(1):
```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].head(1))
```

                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2014-08-01 D      201408 0.001443

```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].tail(1))
```
                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2019-07-26 DUK    201907 0.000167

In [38]:
market_df['YYYYMM'] = market_df.apply(lambda row: row['Date'][:4] + row['Date'][5:7], axis=1)
clean_df['YYYYMM'] = clean_df.apply(lambda row: row['Date'][:4] + row['Date'][5:7], axis=1)
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Return,VolStat,YYYYMM
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000,SPY,0.002846,0.003862,201408
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000,SPY,0.002780,0.005560,201408
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000,SPY,0.004038,0.005451,201408
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000,SPY,0.002059,0.004169,201408
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000,SPY,-0.000752,0.004766,201408


In [39]:
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel,EnhancedVolLevel,YYYYMM
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332,3_HIGH,4_HIGH,201407
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001,2_MEDIUM,3_MEDIUM,201407
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336,3_HIGH,5_VERY_HIGH,201407
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333,3_HIGH,4_HIGH,201407
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000,2_MEDIUM,3_MEDIUM,201408


In [40]:
# Calcular modified_market_df para SPY
modified_market_df = market_df.groupby(['Symbol','YYYYMM']).mean().reset_index()
modified_market_df = modified_market_df.rename(columns={'Return':'MeanMonthSPYReturn'}) # renombrar columna
modified_market_df.head()

,Symbol,YYYYMM,Open,High,Low,Close,Adj Close,Volume,MeanMonthSPYReturn,VolStat
0,SPY,201408,199.212997,199.758000,198.846001,199.499000,180.666200,6.344840e+07,0.001443,0.004579
1,SPY,201409,199.588571,200.292382,198.670477,199.528095,181.012302,1.003077e+08,-0.000285,0.008134
2,SPY,201410,193.549565,194.924348,192.173044,193.614783,176.157063,1.714377e+08,0.000407,0.014307
3,SPY,201411,204.488947,205.043157,203.910000,204.675262,186.220253,8.268904e+07,0.000920,0.005546
4,SPY,201412,205.685455,207.066817,204.780455,205.614546,187.453694,1.384169e+08,-0.000325,0.011195


In [41]:
# Fusión interna added_market_df DataFrame y clean_df por YYYYMM
modified_clean_df = pd.merge(clean_df, modified_market_df[['YYYYMM','MeanMonthSPYReturn']], how='inner', on='YYYYMM')

print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].head(1))
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].tail(1))

         Date Symbol  YYYYMM  MeanMonthSPYReturn
0  2014-08-01      D  201408            0.001443
            Date Symbol  YYYYMM  MeanMonthSPYReturn
5939  2019-07-26    DUK  201907            0.000167


In [42]:
modified_clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel,EnhancedVolLevel,YYYYMM,MeanMonthSPYReturn
0,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000,2_MEDIUM,3_MEDIUM,201408,0.001443
1,2014-08-04,67.480003,67.589996,65.849998,67.029999,54.815529,3911100.0,D,0.025785,-0.006669,3.9111,False,2014,66.823331,3_HIGH,5_VERY_HIGH,201408,0.001443
2,2014-08-05,66.959999,67.089996,65.739998,66.010002,53.981419,3036100.0,D,0.020161,-0.014188,3.0361,False,2014,66.279999,3_HIGH,4_HIGH,201408,0.001443
3,2014-08-06,65.620003,65.699997,64.709999,64.910004,53.081856,5188600.0,D,0.015087,-0.010820,5.1886,False,2014,65.235001,2_MEDIUM,3_MEDIUM,201408,0.001443
4,2014-08-08,66.209999,67.900002,66.209999,67.849998,55.486111,3872800.0,D,0.025525,0.024770,3.8728,True,2014,67.320000,3_HIGH,5_VERY_HIGH,201408,0.001443


## Desglose de los rendimientos en función del rendimiento más amplio del mercado

Ahora podemos comenzar nuestro análisis granular de cómo se relacionan los rendimientos de mercado más amplios con los rendimientos de acciones individuales.

Comencemos por desglosar los rendimientos de mercado más amplios en cuantiles. Este enfoque de análisis de cuantiles que tomaremos se emplea comúnmente en el análisis de datos para determinar cómo se relaciona la magnitud de una variable con otra variable de interés.

Podemos explorar esta idea con el método ```pd.qcut()```. Es decir, ```pd.qcut()``` nos permitirá reducir los rendimientos del mercado por cuantiles (eventualmente los agruparemos por cuantiles) y, por lo tanto, nos permitirá calcular estadísticas de resumen (como el rendimiento promedio) para cada cuantil. .

Primero extraigamos los retornos usando la conveniencia del método ```pivot()``` en un DataFrame. La rotación de un DataFrame se puede lograr especificando:

1. Un índice sobre el que pivotar. En este caso elegimos ```Fecha```.
2. Columnas que nos gustaría tener después del pivote. En este caso, nos gustaría columnas que sean ````Symbol```.
3. Los valores que mostrará cada par (fila,columna). En este caso nos gustaría tener el ```Retorno```.

Haremos pivotar ```merged_df``` usando estas entradas de parámetros para generar un marco de datos donde las filas son la fecha, cada columna es el símbolo y los valores son el rendimiento diario de apertura a cierre para la fecha y el símbolo dados.

In [ ]:
merged_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel,EnhancedVolLevel,Return_SPY
0,2014-08-18,68.970001,69.250000,68.559998,68.680000,56.164867,1375600.0,D,0.010004,-0.004205,1.3756,False,2014,68.829999,1_LOW,2_LOW,0.002846
1,2014-08-18,31.930000,32.099998,31.730000,31.820000,26.593485,4037400.0,EXC,0.011588,-0.003445,4.0374,False,2014,31.883333,1_LOW,2_LOW,0.002846
2,2014-08-18,96.589996,97.180000,95.889999,96.139999,82.166077,1098200.0,NEE,0.013355,-0.004659,1.0982,False,2014,96.403333,2_MEDIUM,3_MEDIUM,0.002846
3,2014-08-18,43.529999,43.720001,43.279999,43.380001,34.210857,2979800.0,SO,0.010108,-0.003446,2.9798,False,2014,43.460000,2_MEDIUM,3_MEDIUM,0.002846
4,2014-08-18,72.300003,72.650002,71.919998,72.089996,58.080738,1826000.0,DUK,0.010097,-0.002905,1.8260,False,2014,72.219999,2_MEDIUM,3_MEDIUM,0.002846


In [43]:
return_df = merged_df.pivot(index='Date', columns='Symbol', values='Return')
return_df.head()

Symbol,D,DUK,EXC,NEE,SO
Date,,,,,
2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446
2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741
2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508
2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000
2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195


In [ ]:
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Return,VolStat,YYYYMM
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000,SPY,0.002846,0.003862,201408
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000,SPY,0.002780,0.005560,201408
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000,SPY,0.004038,0.005451,201408
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000,SPY,0.002059,0.004169,201408
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000,SPY,-0.000752,0.004766,201408


Vamos a fusionar con los rendimientos de mercado más amplios del SPY ```market_df``` cargado anteriormente:

In [44]:
full_df = pd.merge(return_df, market_df[['Date','Return']].set_index('Date'), left_index=True, right_index=True)
full_df = full_df.rename(columns={'Return':'MarketReturn'}).reset_index()
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752


A través de unas pocas líneas simples, hemos creado un DataFrame ```full_df``` donde cada valor es un rendimiento diario de apertura a cierre, ya sea para uno de los cinco símbolos en estudio o para el mercado más amplio.

Procedemos utilizando ```pd.qcut()``` con 10 cuantiles. En general, la cantidad de cuantiles debe elegirse en función de la granularidad de la vista que necesite.

In [45]:
#Cree 10 categorías de cuantiles por el rendimiento del mercado
num_quantiles = 10
full_df['market_quantile'] = pd.qcut(full_df['MarketReturn'],num_quantiles,labels=False)
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn,market_quantile
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846,7
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780,7
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038,7
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059,6
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752,3


Ahora agrupemos por ```market_quantile``` y calculemos el rendimiento medio de todos los símbolos:

In [46]:
# Agrupar por cuantil de mercado y calcule el rendimiento medio
full_df.groupby('market_quantile').mean()

,D,DUK,EXC,NEE,SO,MarketReturn
market_quantile,,,,,,
0,-0.004553,-0.003282,-0.006711,-0.004723,-0.002890,-0.013363
1,-0.002423,-0.001444,-0.002686,-0.001001,-0.000830,-0.005045
2,-0.001612,-0.000617,-0.001087,-0.000693,-0.000061,-0.002577
3,0.001010,0.000420,-0.001277,0.000728,0.000938,-0.001154
4,0.001189,0.000615,0.000377,0.000903,0.000444,-0.000113
5,-0.001516,-0.001656,-0.000699,-0.000985,-0.001240,0.000887
6,0.001950,0.001233,0.002246,0.002210,0.001040,0.002055
7,0.001804,0.000505,0.001006,0.000831,0.000963,0.003568
8,0.002148,0.002366,0.004323,0.003042,0.002563,0.005787


Cada valor en la salida del DataFrame anterior es una media de los rendimientos diarios para un símbolo dado, donde la media se toma en todas las fechas que corresponden al ```market_quantile``` que aparece como el índice del DataFrame de salida. Tenga en cuenta que los números de cuantiles más altos indican rendimientos de mercado más altos, mientras que los números de cuantiles más bajos indican rendimientos de mercado más bajos (en este caso, rendimientos de mercado negativos).

Vemos aquí que los rendimientos de las acciones de energía siguen un patrón cuando los rendimientos de SPY son grandes (número de cuantil más alto) o pequeños (número de cuantil más bajo). Es decir, los rendimientos de las acciones individuales siguen el mismo patrón que los del mercado en general. Por lo tanto, el mercado más amplio tiene un efecto sobre los rendimientos de acciones individuales; es decir, la magnitud del rendimiento más amplio del mercado de valores está correlacionada con la magnitud del rendimiento de las acciones individuales.

### Ejercicio 8:

Creamos cuantiles para los rendimientos del mercado y posteriormente calculamos el rendimiento medio para cada uno de estos cuantiles. Realice un análisis similar al anterior, pero en su lugar agrupe por cuantiles para la volatilidad del mercado en lugar de los rendimientos del mercado, y calcule la desviación estándar de los rendimientos para cada categoría de cuantiles de volatilidad del mercado en lugar del rendimiento medio.

In [48]:
new_df = pd.merge(return_df, market_df[['Date','VolStat']].set_index('Date'), left_index=True, right_index=True)
new_df = new_df.rename(columns={'VolStat':'MarketVolStat'}).reset_index()
#  quantiles
num_quantiles = 10
new_df['market_volstat_quantile'] = pd.qcut(new_df['MarketVolStat'],num_quantiles,labels=False)
# calculo de media o sd
new_df.groupby('market_volstat_quantile').mean()

,D,DUK,EXC,NEE,SO,MarketVolStat
market_volstat_quantile,,,,,,
0,-0.000025,-0.000293,0.000665,0.000453,0.000108,0.003019
1,0.000989,-0.000138,0.000497,0.001039,0.000137,0.004242
2,0.000800,0.000129,-0.000146,0.000650,-0.000208,0.005049
3,0.001307,0.001286,0.000370,0.000934,0.001494,0.005942
4,0.000068,-0.000223,-0.000298,0.000484,0.000024,0.006944
5,-0.000520,-0.000572,0.000196,0.000162,0.000212,0.008085
6,0.000397,-0.001013,-0.000282,0.000551,-0.000731,0.009586
7,0.001017,0.001883,0.001692,0.002217,0.002232,0.011651
8,-0.000315,0.000704,0.000603,-0.000125,0.001274,0.014646


De manera similar a cómo las direcciones de los rendimientos de las acciones siguen las del mercado en general, la volatilidad de los rendimientos de las acciones también sigue la volatilidad de los rendimientos del mercado en general. Esto justifica un análisis más profundo de la causa raíz de este efecto como un proyecto futuro.

## Conclusiones

Hemos explorado los rendimientos bursátiles de las cinco acciones del sector energético en términos de sus propios regímenes de volatilidad y sus rendimientos y volatilidad en relación con el mercado en general. Encontramos que para las acciones bajo estudio, no existe un vínculo fuerte entre el nivel de volatilidad y la dirección del rendimiento diario de las acciones. Además, descubrimos que al comparar acciones con el mercado en general, sus rendimientos y niveles de volatilidad se amplifican cuando los rendimientos y los niveles de volatilidad del mercado son altos. Estos hallazgos indican que existe un vínculo intrínseco entre los rendimientos y la volatilidad, tanto en el caso de acciones individuales como en el caso de mercado más amplio. Esto brinda una variedad de vías de exploración para proyectos de seguimiento.

## Para llevar

En este caso, aprendió múltiples herramientas de manipulación de datos en ```pandas```, incluidas funciones anónimas, agrupación, merge, corte de cuantiles y pivoteo, mientras utilizaba técnicas de análisis de agregación y transformación de datos que aprendimos anteriormente.

```pandas``` es un paquete increíblemente versátil y puede aumentar significativamente la productividad y brindar información empresarial excepcional. Estas técnicas deberían servir como una base sólida para cualquier análisis de datos futuro que pueda realizar.